#### Import needed

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import pickle as pkl
from datasets import Dataset
from random import uniform
from time import sleep

#### Getting all the links of the pages

In [46]:
last_page = 4150
link_list = []
for i in range(1, last_page+1):
    link_list.append("https://stats.stackexchange.com/questions?tab=votes&page={}".format(i))

#### Getting all the links of the questions

In [ ]:
questions = []
start = 0
end = 500
for link in tqdm(link_list[start:end]):
    page = requests.get(link)
    if page.status_code == 200:
        pageParsed = BeautifulSoup(page.content, 'html.parser')
        try:
            all_page = pageParsed.find_all('div', {'class':'s-post-summary--content'})
            for question in all_page:
                question_link = question.find('h3', class_='s-post-summary--content-title').find('a')['href']
                questions.append('https://stats.stackexchange.com' + question_link)
        except:
            print('Failed')

#### Save the links list of the questions

In [63]:
with open('questions_links.pkl', 'wb') as file:
    pkl.dump(questions, file)

#### Open the list

In [5]:
with open('questions_links.pkl', 'rb') as file:
    questions_links = pkl.load(file)
print(len(questions_links))

25000


#### Create a dictionary that contains:

- index

- question

- answer

In [48]:
question_link = "https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw"
page = requests.get(question_link)
pageParsed = BeautifulSoup(page.content, 'html.parser')
print(pageParsed.find('div', {'class': 'd-flex sm:fd-column'}).find('a').text)

How to choose the number of hidden layers and nodes in a feedforward neural network?


In [49]:
def scrape(df_dict, idx, start, stop, save=True):
    
    # Set headers and user agent
    headers = {'User-Agent': 
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'}
    page_num = start
    for question_link in tqdm(questions_links[start:stop]):
        sleep(uniform(0.5, 1.0))

        try:
            page = requests.get(question_link, headers=headers)
            pageParsed = BeautifulSoup(page.content, 'html.parser')

            question = pageParsed.find('div', {'class': 'd-flex sm:fd-column'}).find('a').text
            texts = pageParsed.find_all('div', {'class': 's-prose js-post-body'})

            if len(texts) > 1:
                #question = texts[0].text.strip()

                for answer in texts[1:]:
                    df_dict[idx] = {'question': question, 'answer': answer.text.strip()}
                    idx += 1
        except:
            print(f'Failed page: {page_num}')
            
        page_num += page_num

    if save == True:
        with open('df_dict.pkl', 'wb') as file:
            pkl.dump(df_dict, file)
            
    return df_dict, idx

In [52]:
df_dict = {}
idx = 1
start = 0
stop = 10000
df_dict, idx = scrape(df_dict=df_dict, idx=idx, start=start, stop=stop, save=True)

100%|██████████| 10000/10000 [5:42:31<00:00,  2.06s/it]  


In [53]:
def scrape2(df_dict, idx, start, stop, save=True):
    
    # Set headers and user agent
    headers = {'User-Agent': 
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'}
    page_num = start
    for question_link in tqdm(questions_links[start:stop]):
        sleep(uniform(0.4, 0.6))

        try:
            page = requests.get(question_link, headers=headers)
            pageParsed = BeautifulSoup(page.content, 'html.parser')

            question = pageParsed.find('div', {'class': 'd-flex sm:fd-column'}).find('a').text
            texts = pageParsed.find_all('div', {'class': 's-prose js-post-body'})

            if len(texts) > 1:
                #question = texts[0].text.strip()

                for answer in texts[1:]:
                    df_dict[idx] = {'question': question, 'answer': answer.text.strip()}
                    idx += 1
        except:
            print(f'Failed page: {page_num}')
            
        page_num += page_num

    if save == True:
        with open('df_dict2.pkl', 'wb') as file:
            pkl.dump(df_dict, file)
            
    return df_dict, idx

In [54]:
df_dict2 = {}
idx = 100000
start = 10000
stop = 25000
df_dict2, idx = scrape2(df_dict=df_dict2, idx=idx, start=start, stop=stop, save=True)

  7%|▋         | 1048/15000 [18:06<3:42:37,  1.04it/s] 

Failed page: 15080143012651102122105311357894339192625630629371264454491537662404055861234318233427433945345664359270563562536002092912033851249947657947764798994417377555483387729269740087126710097647697950991970872765238960725218615191034428672351193103876676882863794007328060981060073868574984560324723087895364035912432353280000


  7%|▋         | 1049/15000 [18:07<3:24:00,  1.14it/s]

Failed page: 30160286025302204244210622715788678385251261258742528908983075324808111722468636466854867890691328718541127125072004185824067702499895315895529597988834755110966775458539480174253420195295395901983941745530477921450437230382068857344702386207753353765727588014656121962120147737149969120649446175790728071824864706560000


  7%|▋         | 1050/15000 [18:07<3:13:28,  1.20it/s]

Failed page: 60320572050604408488421245431577356770502522517485057817966150649616223444937272933709735781382657437082254250144008371648135404999790631791059195977669510221933550917078960348506840390590791803967883491060955842900874460764137714689404772415506707531455176029312243924240295474299938241298892351581456143649729413120000


100%|██████████| 15000/15000 [4:22:57<00:00,  1.05s/it]   


#### Create a dataframe and then a transformers Dataset from the dictionary

In [27]:
rows = [{'idx': key, 'question': value['question'], 'answer': value['answer']} for key, value in df_dict.items()]
df = pd.DataFrame(rows)
df['conversation'] = df['question'] + '\n\n' + df['answer']
print(df.shape)
display(df.head())
dataset = Dataset.from_pandas(df)

(623, 4)


,idx,question,answer,conversation
0,1,In today's pattern recognition class my profes...,Imagine a big family dinner where everybody st...,In today's pattern recognition class my profes...
1,2,In today's pattern recognition class my profes...,"The manuscript ""A tutorial on Principal Compon...",In today's pattern recognition class my profes...
2,3,In today's pattern recognition class my profes...,Let's do (2) first. PCA fits an ellipsoid to ...,In today's pattern recognition class my profes...
3,4,In today's pattern recognition class my profes...,"Hmm, here goes for a completely non-mathematic...",In today's pattern recognition class my profes...
4,5,In today's pattern recognition class my profes...,"I'd answer in ""layman's terms"" by saying that ...",In today's pattern recognition class my profes...


#### Save the dataset

In [134]:
with open('df_dict.pkl', 'wb') as file:
    pkl.dump(df_dict, file)

In [ ]:
with open('dataset.pkl', 'wb') as file:
    pkl.dump(dataset, file)